In [ ]:
%matplotlib inline


Created on Thu Feb  6 17:04:59 2020

@author: makinea1



In [ ]:
import sys
path = '/m/home/home8/80/makinea1/unix/pythonstuff/bfieldtools'
if path not in sys.path:
    sys.path.insert(0, path)

import numpy as np
import trimesh
from bfieldtools.mesh_class import Conductor, StreamFunction
from mayavi import mlab
from matplotlib import pyplot as plt


import pkg_resources
#Load simple plane mesh that is centered on the origin
file_obj = pkg_resources.resource_filename('bfieldtools',
                'example_meshes/10x10_plane_hires.obj')
mesh = trimesh.load(file_obj, process=True)
t = np.eye(4)
t[1:3,1:3] = np.array([[0,1],[-1,0]])
mesh.apply_transform(t)


#x = np.sin(np.pi/6)
#y = np.cos(np.pi/6)
#points = np.array([[0, 0, 0],
#                   [1, 0, 0],
#                   [x, y, 0],
#                   [-x, y, 0],
#                   [-1, 0, 0],
#                   [-x, -y, 0],
#                   [x, -y, 0]])
#
#tris = np.array([[0,1,2],[0,2,3],[0,3,4],[0,4,5],[0,5,6],[0,6,1]])
#mesh = trimesh.Trimesh(points, tris)
#scalars = np.zeros(7)
#scalars[0] = 1

cond = Conductor(mesh_obj=mesh)
scalars = np.zeros(len(mesh.vertices))
scalars[cond.inner_vertices] = 1
s = StreamFunction(scalars, cond)

N = 100
x = y = np.linspace(-1.8, 1.8, N)
X,Y = np.meshgrid(x, y, indexing='ij')
points = np.zeros((X.flatten().shape[0], 3))
points[:, 0] = X.flatten()
points[:, 1] = Y.flatten()

#points = np.zeros((100, 3))
#points[:, 2] = np.linspace(0.1, 10, 100)

s.plot(False)

np = 5
fig, ax = plt.subplots(1, np)
for nn in range(np):
    plt.sca(ax[nn])

    points[:, 2] = 0.2*(nn+1)

    mlab.points3d(*points.T)

    cond.U_coupling.reset()
    U1 = cond.U_coupling(points).copy()
    U1_sum = (U1 @ scalars).reshape(N, N)
#    plt.figure()
#    plt.imshow(U1_sum)
#    plt.colorbar()

    cond.U_coupling.reset()
    U2 = cond.U_coupling(points, approx_far=True, margin=0).copy()
    U2_sum = (U2 @ scalars).reshape(N, N)
#    plt.figure()
#    plt.imshow(U2_sum)
#    plt.colorbar()

#    plt.figure()
    plt.imshow(abs(U2_sum-U1_sum)/abs(U1_sum)*100)
    plt.colorbar()

#plt.plot(abs((U1_sum-U2_sum)/U1_sum.max()))